In [ ]:
!pip install nltk
!pip install emoji
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.6 MB/s eta 0:00:00


#Importando o Dataset

In [ ]:
import pandas as pd
import numpy as np
url = 'https://raw.githubusercontent.com/VictorHNascimento/ML-Olympiad-Toxic-Language-PTBR-Detection/main/toxic_language_test.csv'
!wget {url} -O dataset.csv
df = pd.read_csv('dataset.csv')
df.head()

--2024-05-30 22:53:07--  https://raw.githubusercontent.com/VictorHNascimento/ML-Olympiad-Toxic-Language-PTBR-Detection/main/toxic_language_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 405996 (396K) [text/plain]
Saving to: ‘dataset.csv’

dataset.csv         100%[===================>] 396.48K  --.-KB/s    in 0.05s   

2024-05-30 22:53:07 (8.44 MB/s) - ‘dataset.csv’ saved [405996/405996]



,id,text
0,0,@user nossa mano te odeio na moral kkkkkkkkkkk...
1,1,@user @user a edição ficou muito boa! kkkkkkkk...
2,2,"@user largada vá, visto que a vaca anda à solta 😂"
3,3,"poxa, eu queria ganhar um boné. alguém me da u..."
4,4,"@user amiga, tudo bem, não precisa se desculpa..."


In [ ]:
df.shape[0]

4200

# Funções de Pre-Processamento

In [ ]:
import re
import emoji
from unidecode import unidecode

def limpaTexto(text):
    # Remover emojis
    text = emoji.replace_emoji(text, replace='')  # Remove emojis

    # Remover menções de usuários
    text = re.sub(r'@\w+', ' ', text)

    # Remover links
    text = re.sub(r'https?://\S+', ' ', text)

    # Remover espaços extras
    text = re.sub(r'\s+', ' ', text)

    # Remover palavras específicas
    text = re.sub(r'\b(rt|user|https)\b', ' ', text, flags=re.IGNORECASE)

    # Remover acentuação
    text = unidecode(text)

    return text.strip()

def defineListas(df):
  #Criando Lista de Palavras
  toxic = set(''.join(df[df['label'] == 1]['text']).split())
  nontoxic = set(''.join(df[df['label'] == 0]['text']).split())
  return toxic, nontoxic

def contaToxicos(text, toxic_words):
  count = 0
  for p in text.split():
    if p in toxic_words:
      count += 1
  return count

def contaNaotoxicos(text, nontoxic_words):
  count = 0
  for p in text.split():
    if p in nontoxic_words:
      count += 1
  return count

def contaNeutros(text, toxic_words, nontoxic_words):
  words = text.split()
  count = 0
  for p in text.split():
    if p in nontoxic_words and p in toxic_words:
      count += 1
  return count

def contemToxicos(text, toxic_words):
  for p in text.split():
    if p in toxic_words:
      return 1
  return 0

def contemNaotoxicos(text, nontoxic_words):
  for p in text.split():
    if p in nontoxic_words:
      return 1
  return 0

def contemNeutros(text, toxic_words, nontoxic_words):
  for p in text.split():
     if p in nontoxic_words and p in toxic_words:
      return 1
  return 0

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

#Obter as stopwords em PT-BR
stop_words = set(stopwords.words('portuguese'))

def removerStopwords(text):
  tokens = text.split()
  tokens_filtrados = [word for word in tokens if word.lower() not in stop_words]
  return ' '.join(tokens_filtrados)

df['text'] = df['text'].apply(removerStopwords)

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

def lemmatizaTexto(text):
  lemmatizer= WordNetLemmatizer()

  # Tokenizar o texto em palavras
  tokens = word_tokenize(text, language='portuguese')

  # Lematizar cada palavra
  lemmatized_tokens= [lemmatizer.lemmatize(word) for word in tokens]

  # Reunir as palavras lematizadas em uma string novamente
  return' '.join(lemmatized_tokens)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


#Pre Processando os Dados

In [ ]:
# Aplicar a função de limpeza do texto
df['text'] = df['text'].apply(limpaTexto)

# Visualizar os tweets processados
df[['text']].head()

,text
0,mano odeio moral kkkkkkkkkkkkkkkkkkk
1,"edicao ficou boa! kkkkkkkkkkk faltou ""luka time""."
2,"largada va, visto vaca anda solta"
3,"poxa, queria ganhar bone. alguem bone? :((("
4,"amiga, tudo bem, precisa desculpar ele. escrot..."


In [ ]:
#Removendo Stop Words
df['text'] = df['text'].apply(removerStopwords)
df['text'].head()

0                 mano odeio moral kkkkkkkkkkkkkkkkkkk
1    edicao ficou boa! kkkkkkkkkkk faltou "luka time".
2                    largada va, visto vaca anda solta
3          poxa, queria ganhar bone. alguem bone? :(((
4    amiga, tudo bem, precisa desculpar ele. escrot...
Name: text, dtype: object

In [ ]:
# Lematizando o Texto
df['text'] = df['text'].apply(lemmatizaTexto)

In [ ]:
# Importando o Dataset Tratado
url = 'https://raw.githubusercontent.com/VictorHNascimento/ML-Olympiad-Toxic-Language-PTBR-Detection/main/toxic_language_train.csv'
!wget {url} -O temp_palavras.csv
df_palavras = pd.read_csv('temp_palavras.csv')

# Aplicar a função de limpeza do texto
df_palavras['text'] = df_palavras['text'].apply(limpaTexto)
df_palavras['text'] = df_palavras['text'].apply(removerStopwords)
df_palavras['text'] = df_palavras['text'].apply(lemmatizaTexto)

#Criando as Listas
pal_toxicas = {}
pal_naotoxicas = {}

pal_toxicas, pal_naotoxicas = defineListas(df_palavras)

--2024-05-30 22:53:10--  https://raw.githubusercontent.com/VictorHNascimento/ML-Olympiad-Toxic-Language-PTBR-Detection/main/toxic_language_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1573871 (1.5M) [text/plain]
Saving to: ‘temp_palavras.csv’

temp_palavras.csv   100%[===================>]   1.50M  --.-KB/s    in 0.07s   

2024-05-30 22:53:11 (22.4 MB/s) - ‘temp_palavras.csv’ saved [1573871/1573871]



In [ ]:
#Criando Novas Features
df['cont_toxic_words'] = df['text'].apply(lambda x: contaToxicos(x, pal_toxicas))
df['cont_nontoxic_words'] = df['text'].apply(lambda x: contaNaotoxicos(x, pal_naotoxicas))
df['cont_neutros'] = df['text'].apply(lambda x: contaNeutros(x, pal_toxicas, pal_naotoxicas))
df['toxic_words'] = df['text'].apply(lambda x: contemToxicos(x, pal_toxicas))
df['non_toxic_words'] = df['text'].apply(lambda x: contemNaotoxicos(x, pal_naotoxicas))
df['neutral_words'] = df['text'].apply(lambda x: contemNeutros(x, pal_toxicas, pal_naotoxicas))
df['char_cont'] = df['text'].apply(lambda x: len(x))
df['words_cont'] = df['text'].apply(lambda x: len(x.split()))
df.head()

,id,text,cont_toxic_words,cont_nontoxic_words,cont_neutros,toxic_words,non_toxic_words,neutral_words,char_cont,words_cont
0,0,mano odeio moral kkkkkkkkkkkkkkkkkkk,4,4,4,1,1,1,36,4
1,1,edicao ficou boa ! kkkkkkkkkkk faltou `` luka ...,10,10,10,1,1,1,55,11
2,2,"largada va , visto vaca anda solta",7,7,7,1,1,1,34,7
3,3,"poxa , queria ganhar bone . alguem bone ? : ( ( (",11,13,11,1,1,1,49,13
4,4,"amiga , tudo bem , precisa desculpar ele . esc...",14,13,13,1,1,1,67,14


#Funções de Transformação/Modelagem

In [ ]:
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

def TFIDF(df_tratado):

  df = df_tratado
  df = df.dropna() #Dropando Valores Nulos

  #Separando a coluna de Texto
  x_text = df.iloc[:, 1]
  x_text.head()

  #Separando as Features
  x_features = df.iloc[ :, 2:]
  x_features.head()

  #Vetorizando o texto
  url = 'https://github.com/VictorHNascimento/ML-Olympiad-Toxic-Language-PTBR-Detection/raw/main/vectorizer.pkl'
  !wget {url} -O tfidf_vectorizer.pkl
  vectorizer = joblib.load('tfidf_vectorizer.pkl')
  x_text = vectorizer.transform(x_text)

  x = hstack([x_text, x_features]) #Juntando texto vetorizado com as features

  return x

#Transformação e Predição

In [ ]:
#Vetorizando o texto
x = TFIDF(df)
x.shape

--2024-05-30 22:53:18--  https://github.com/VictorHNascimento/ML-Olympiad-Toxic-Language-PTBR-Detection/raw/main/vectorizer.pkl
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/VictorHNascimento/ML-Olympiad-Toxic-Language-PTBR-Detection/main/vectorizer.pkl [following]
--2024-05-30 22:53:19--  https://raw.githubusercontent.com/VictorHNascimento/ML-Olympiad-Toxic-Language-PTBR-Detection/main/vectorizer.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 663246 (648K) [application/octet-stream]
Saving to: ‘tfidf_vectorizer.pkl’

tfidf_vectorizer.pk 100%[===================>] 647.70K  --.-KB/s    in 

(4200, 22578)

In [ ]:
#Importando o modelo
url = 'https://github.com/VictorHNascimento/ML-Olympiad-Toxic-Language-PTBR-Detection/raw/main/identifica_toxicos.pkl'
!wget {url} -O modelo.pkl
model = joblib.load('modelo.pkl')

--2024-05-30 22:53:19--  https://github.com/VictorHNascimento/ML-Olympiad-Toxic-Language-PTBR-Detection/raw/main/identifica_toxicos.pkl
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/VictorHNascimento/ML-Olympiad-Toxic-Language-PTBR-Detection/main/identifica_toxicos.pkl [following]
--2024-05-30 22:53:19--  https://raw.githubusercontent.com/VictorHNascimento/ML-Olympiad-Toxic-Language-PTBR-Detection/main/identifica_toxicos.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 364175 (356K) [application/octet-stream]
Saving to: ‘modelo.pkl’

modelo.pkl          100%[===================>] 355.64K  -

In [ ]:
#Predizendo os resultados
predicao = model.predict(x)

In [ ]:
#Inserindo no DataFrame
df['label'] = predicao

# Salvar o DataFrame com as previsões em um novo arquivo CSV
df.to_csv('toxic_language_test_predicao.csv', index=False)

In [ ]:
df.head(20)

,id,text,cont_toxic_words,cont_nontoxic_words,cont_neutros,toxic_words,non_toxic_words,neutral_words,char_cont,words_cont,label
0,0,mano odeio moral kkkkkkkkkkkkkkkkkkk,4,4,4,1,1,1,36,4,0
1,1,edicao ficou boa ! kkkkkkkkkkk faltou `` luka ...,10,10,10,1,1,1,55,11,0
2,2,"largada va , visto vaca anda solta",7,7,7,1,1,1,34,7,0
3,3,"poxa , queria ganhar bone . alguem bone ? : ( ( (",11,13,11,1,1,1,49,13,0
4,4,"amiga , tudo bem , precisa desculpar ele . esc...",14,13,13,1,1,1,67,14,1
5,5,Entao pra justo caia pau vacilar campo politic...,40,38,38,1,1,1,212,41,1
6,6,"minha , cuido mesmo , pronto acabou",6,7,6,1,1,1,35,7,0
7,7,ante tava bem melhor,4,4,4,1,1,1,20,4,0
8,8,"`` amigo , seguir reto aqui caio dutra ? ''",9,9,9,1,1,1,43,10,0
9,9,"mano , mentira coisa tao feia tao feia pqp , q...",18,18,18,1,1,1,84,18,1


#Montando Submissão Kaggle

In [ ]:
# Carregar o arquivo CSV (baixado do kaggle)
df_sub = pd.read_csv('sample_submission.csv')

# Remover a coluna label do template
df_sub = df_sub.drop(columns=['label'])

# Adicionar a coluna prediction_label de df_sub
df_sub['label'] = df['label']

# Salvar o DataFrame com as previsões em um novo arquivo CSV
df_sub.to_csv('submissao_final.csv', index=False)

# Exibir o DataFrame final_df atualizado
df_sub.head()

,id,label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,1
